#About

This notebook contains the SHiP Tracker data preparation.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Data Preparation

In [2]:
path_to_data = '/Users/mikhail91/Documents/ShiP-Tracking/Data/ship.10.0.Pythia8-TGeant4.root'

Open root file

In [3]:
import ROOT
import root_numpy
f=ROOT.TFile(path_to_data)
tree=f.Get("cbmsim")
leaves = tree.GetListOfLeaves()

TClass::TClass:0: RuntimeWarning: no dictionary for class ShipMCTrack is available
TClass::TClass:0: RuntimeWarning: no dictionary for class vetoPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class FairMCPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class FairMultiLinkedData_Interface is available
TClass::TClass:0: RuntimeWarning: no dictionary for class ShipRpcPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class TargetPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class strawtubesPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class ecalPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class hcalPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class muonPoint is available
TClass::TClass:0: RuntimeWarning: no dictionary for class FairMCEventHeader is available
TClass::TClass:0: RuntimeWarning: no dictionary for class FairFile

Get names of all leaves in the file.

In [4]:
leave_names = []
for i in range(0,leaves.GetEntries() ) :
    leaf = leaves.At(i)
    name = leaf.GetName()
    leave_names.append(name)

Find leaves for 'strawtubesPoint' branch.

In [5]:
import re
f = re.compile('[a-zA-Z0-9._ ]*strawtubesPoint[a-zA-Z0-9._ ]*')
sel_leaves = []
for i in leave_names:
    res = f.findall(i)
    if res==[]:
        pass
    else:
        sel_leaves.append(res[0])

Get branch 'strawtubesPoint'.

In [6]:
data_root = root_numpy.root2array(path_to_data, 'cbmsim', branches=sel_leaves)
data = pd.DataFrame(data_root)
data.irow(range(0,10))

,cbmroot.strawtubes.strawtubesPoint_,cbmroot.strawtubes.strawtubesPoint.fUniqueID,cbmroot.strawtubes.strawtubesPoint.fBits,cbmroot.strawtubes.strawtubesPoint.fRefToLinks,cbmroot.strawtubes.strawtubesPoint.fTrackID,cbmroot.strawtubes.strawtubesPoint.fEventId,cbmroot.strawtubes.strawtubesPoint.fPx,cbmroot.strawtubes.strawtubesPoint.fPy,cbmroot.strawtubes.strawtubesPoint.fPz,cbmroot.strawtubes.strawtubesPoint.fTime,cbmroot.strawtubes.strawtubesPoint.fLength,cbmroot.strawtubes.strawtubesPoint.fELoss,cbmroot.strawtubes.strawtubesPoint.fDetectorID,cbmroot.strawtubes.strawtubesPoint.fX,cbmroot.strawtubes.strawtubesPoint.fY,cbmroot.strawtubes.strawtubesPoint.fZ,cbmroot.strawtubes.strawtubesPoint.fPdgCode,cbmroot.strawtubes.strawtubesPoint.fdist2Wire
0,0,0,50331648,0,3,0,-0.358307,-0.329753,15.881699,232.044861,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822
1,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
2,0,0,33554432,0,3,0,-0.239095,0.152370,25.827625,383.923828,0.559879,0.000002,10012283,64.405243,3.918991,2582.247559,-13,0.398998
3,0,0,33554432,0,6,0,0.017466,-0.022435,0.111611,384.985504,0.978458,0.000004,10012334,81.981796,-86.299446,2582.238037,11,0.060633
4,0,0,33554432,0,3,0,-0.254975,-0.028889,17.733240,384.496338,0.824419,0.000002,10002268,-247.450851,29.300148,2581.150391,-13,0.260149
5,0,0,33554432,0,3,0,0.413617,-0.486352,33.006119,231.903000,0.730933,0.000003,50002301,34.251995,-29.362476,-1974.854736,13,0.175176
6,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
7,0,0,33554432,0,3,0,0.772509,0.055323,33.178158,383.944519,0.944866,0.000004,10012268,130.437469,29.799639,2582.250244,-13,0.120362
8,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
9,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000


Remane the columns.

In [7]:
m = re.compile('(?<=cbmroot.strawtubes.strawtubesPoint.f)\w+')
new_cols = []
for i in data.columns:
    res = m.findall(i)
    if res==[]:
        pass
    else:
        new_cols.append(res[0])

In [8]:
data.columns = ['_'] + new_cols
data.irow(range(0,10))

,_,UniqueID,Bits,RefToLinks,TrackID,EventId,Px,Py,Pz,Time,Length,ELoss,DetectorID,X,Y,Z,PdgCode,dist2Wire
0,0,0,50331648,0,3,0,-0.358307,-0.329753,15.881699,232.044861,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822
1,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
2,0,0,33554432,0,3,0,-0.239095,0.152370,25.827625,383.923828,0.559879,0.000002,10012283,64.405243,3.918991,2582.247559,-13,0.398998
3,0,0,33554432,0,6,0,0.017466,-0.022435,0.111611,384.985504,0.978458,0.000004,10012334,81.981796,-86.299446,2582.238037,11,0.060633
4,0,0,33554432,0,3,0,-0.254975,-0.028889,17.733240,384.496338,0.824419,0.000002,10002268,-247.450851,29.300148,2581.150391,-13,0.260149
5,0,0,33554432,0,3,0,0.413617,-0.486352,33.006119,231.903000,0.730933,0.000003,50002301,34.251995,-29.362476,-1974.854736,13,0.175176
6,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
7,0,0,33554432,0,3,0,0.772509,0.055323,33.178158,383.944519,0.944866,0.000004,10012268,130.437469,29.799639,2582.250244,-13,0.120362
8,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
9,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000


Drop not informative columns

In [9]:
data.drop(['_', 'UniqueID', 'RefToLinks', 'EventId'], axis=1, inplace=True)
data.irow(range(0,10))

,Bits,TrackID,Px,Py,Pz,Time,Length,ELoss,DetectorID,X,Y,Z,PdgCode,dist2Wire
0,50331648,3,-0.358307,-0.329753,15.881699,232.044861,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822
1,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
2,33554432,3,-0.239095,0.152370,25.827625,383.923828,0.559879,0.000002,10012283,64.405243,3.918991,2582.247559,-13,0.398998
3,33554432,6,0.017466,-0.022435,0.111611,384.985504,0.978458,0.000004,10012334,81.981796,-86.299446,2582.238037,11,0.060633
4,33554432,3,-0.254975,-0.028889,17.733240,384.496338,0.824419,0.000002,10002268,-247.450851,29.300148,2581.150391,-13,0.260149
5,33554432,3,0.413617,-0.486352,33.006119,231.903000,0.730933,0.000003,50002301,34.251995,-29.362476,-1974.854736,13,0.175176
6,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
7,33554432,3,0.772509,0.055323,33.178158,383.944519,0.944866,0.000004,10012268,130.437469,29.799639,2582.250244,-13,0.120362
8,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000
9,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000


The detectorID decoding.

The station number (statnb = 1,2,3,4; for the Veto station statnb = 5),

The viewnumber (vnb = 0,1,2,3),

The planenumber (pnb = 0,1),

The layernumber (lnb = 0,1),

The strawnumber (snb)

In [10]:
detID = data.DetectorID.values.astype(np.int32)
StatNb = []
ViewNb = []
PlaneNb = []
LayerNb = []
StrawNb = []
WireX = []
WireY = []
WireZ = []
for i in detID:
    statnb = i//10000000
    vnb = (i - statnb*10000000)//1000000
    pnb = (i - statnb*10000000 - vnb*1000000)//100000
    lnb = (i - statnb*10000000 - vnb*1000000 - pnb*100000)//10000
    snb = i - statnb*10000000 - vnb*1000000 - pnb*100000 - lnb*10000 - 2000
    
    StatNb.append(statnb)
    ViewNb.append(vnb)
    PlaneNb.append(pnb)
    LayerNb.append(lnb)
    StrawNb.append(snb)
    
data['StatNb'] = StatNb
data['ViewNb'] = ViewNb
data['PlaneNb'] = PlaneNb
data['LayerNb'] = LayerNb
data['StrawNb'] = StrawNb

In [11]:
data.irow(range(0,10))

,Bits,TrackID,Px,Py,Pz,Time,Length,ELoss,DetectorID,X,Y,Z,PdgCode,dist2Wire,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb
0,50331648,3,-0.358307,-0.329753,15.881699,232.044861,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822,5,0,0,0,319
1,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0,0,0,0,-2000
2,33554432,3,-0.239095,0.152370,25.827625,383.923828,0.559879,0.000002,10012283,64.405243,3.918991,2582.247559,-13,0.398998,1,0,0,1,283
3,33554432,6,0.017466,-0.022435,0.111611,384.985504,0.978458,0.000004,10012334,81.981796,-86.299446,2582.238037,11,0.060633,1,0,0,1,334
4,33554432,3,-0.254975,-0.028889,17.733240,384.496338,0.824419,0.000002,10002268,-247.450851,29.300148,2581.150391,-13,0.260149,1,0,0,0,268
5,33554432,3,0.413617,-0.486352,33.006119,231.903000,0.730933,0.000003,50002301,34.251995,-29.362476,-1974.854736,13,0.175176,5,0,0,0,301
6,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0,0,0,0,-2000
7,33554432,3,0.772509,0.055323,33.178158,383.944519,0.944866,0.000004,10012268,130.437469,29.799639,2582.250244,-13,0.120362,1,0,0,1,268
8,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0,0,0,0,-2000
9,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.000000,0,0,0,0,-2000


Write the data.

In [12]:
#data.to_csv('Data/ship.10.0.Pythia8-TGeant4.strawtubesPoint.csv')